## **Entregable 1 - Fernando Martínez (comisión Jueves)**

In [1]:
# Instalo las bibliotecas necesarias
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Importo las bibliotecas necesarias:
import requests
import json
import pandas as pd
import psycopg2
from io import StringIO

In [16]:
# Importo las bibliotecas necesarias:
import requests
import json
import pandas as pd
import psycopg2
from io import StringIO

# Nueva URL de la API para obtener los datos diarios de COVID-19 para Estados Unidos
url = 'https://api.covidtracking.com/v1/us/daily.json'

# Realizo la solicitud HTTP a la API
response = requests.get(url)

# Obtengo los datos en formato JSON
data = response.json()

# Selecciono las columnas relevantes (al menos 10 variables)
columnas = ['date', 'positive', 'death', 'positiveIncrease', 'deathIncrease', 'totalTestResults', 'hospitalizedCurrently', 'recovered', 'total', 'totalTestResultsIncrease']
datos = []

# Extraigo los datos para cada día
for registro in data:
    registro_seleccionado = {columna: registro[columna] for columna in columnas if columna in registro}
    datos.append(registro_seleccionado)

# Creo el dataframe de Pandas
df = pd.DataFrame(datos)

# Renombro las columnas
nombres_columnas = {
    'date': 'submission_date',
    'positive': 'tot_cases',
    'death': 'tot_death',
    'positiveIncrease': 'new_case',
    'deathIncrease': 'new_death',
    'totalTestResults': 'total_test_results',
    'hospitalizedCurrently': 'hospitalized_currently',
    'recovered': 'recovered',
    'total': 'total',
    'totalTestResultsIncrease': 'total_test_results_increase'
}
df = df.rename(columns=nombres_columnas)

# Muestro el dataframe
print(df)

     submission_date   tot_cases  tot_death  new_case  new_death  \
0           20210307  28756489.0   515151.0     41835        842   
1           20210306  28714654.0   514309.0     60015       1680   
2           20210305  28654639.0   512629.0     68787       2221   
3           20210304  28585852.0   510408.0     65487       1743   
4           20210303  28520365.0   508665.0     66836       2449   
..               ...         ...        ...       ...        ...   
415         20200117         0.0        NaN         0          0   
416         20200116         0.0        NaN         0          0   
417         20200115         0.0        NaN         0          0   
418         20200114         0.0        NaN         0          0   
419         20200113         NaN        NaN         0          0   

     total_test_results  hospitalized_currently recovered  total  \
0             363825123                 40199.0      None      0   
1             362655064                 41401.0

In [30]:
import psycopg2
from io import StringIO

# Conexión a Amazon Redshift
host = 'data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com'
port = 5439
database = 'data-engineer-database'
user = 'fgmartinez87_coderhouse'
password = '7c92hMs3M1'

# Creo la conexión a Amazon Redshift
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)



In [31]:
# Creo la tabla en Redshift
create_table_query = '''
CREATE TABLE IF NOT EXISTS covid_data (
    submission_date DATE,
    tot_cases INT,
    tot_death INT,
    new_case INT,
    new_death INT,
    total_test_results INT,
    hospitalized_currently INT,
    recovered INT,
    total INT,
    total_test_results_increase INT
);
'''
with conn.cursor() as cur:
    cur.execute(create_table_query)
    conn.commit()

In [36]:
# Realizo la consulta en Redshift para ver si se creo la tabla
query = 'SELECT * FROM covid_data;'
df_result = pd.read_sql_query(query, conn)

# Muestro el resultado
print(df_result)

<ipython-input-36-c7ea476a5afb>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql_query(query, conn)


Empty DataFrame
Columns: [submission_date, tot_cases, tot_death, new_case, new_death, consent_cases, consent_deaths, conf_death, conf_cases, pnew_case]
Index: []



## **Entregable 2 - Fernando Martínez (comisión Jueves)**

In [37]:
# Inserto los datos del df
insert_query = 'INSERT INTO covid_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
data_to_insert = [tuple(row) for row in df.values]
with conn.cursor() as cur:
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

In [38]:
# Realizo la consulta en Redshift para ver si se cargó la data
query = 'SELECT * FROM covid_data;'
df_result = pd.read_sql_query(query, conn)

# Muestro el resultado
print(df_result)

<ipython-input-38-c7ea476a5afb>:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_result = pd.read_sql_query(query, conn)


    submission_date   tot_cases   tot_death  new_case  new_death  \
0        1970-01-01    28756489      515151   41835.0      842.0   
1        1970-01-01    28714654      514309   60015.0     1680.0   
2        1970-01-01    28654639      512629   68787.0     2221.0   
3        1970-01-01    28585852      510408   65487.0     1743.0   
4        1970-01-01    28520365      508665   66836.0     2449.0   
..              ...         ...         ...       ...        ...   
415      1970-01-01           0 -2147483648       0.0        0.0   
416      1970-01-01           0 -2147483648       0.0        0.0   
417      1970-01-01           0 -2147483648       0.0        0.0   
418      1970-01-01           0 -2147483648       0.0        0.0   
419      1970-01-01 -2147483648 -2147483648       0.0        0.0   

    consent_cases consent_deaths conf_death  conf_cases  pnew_case  
0       363825123        40199.0       None         0.0  1170059.0  
1       362655064        41401.0       None  

In [39]:
# Cuento la cantidad de registros para validar
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM covid_data")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data: {count}")

Cantidad de registros en la tabla covid_data: 420


In [40]:
# Quiero validar que no hayan duplicados
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM (SELECT DISTINCT * FROM covid_data) AS unique_records;")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros únicos en la tabla covid_data: {count}")

Cantidad de registros únicos en la tabla covid_data: 413


**Existen** resgistros duplicado

In [ ]:
# Query para borrar la tabla
# with conn.cursor() as cur:
#    truncate_query = 'DROP TABLE covid_data;'
#    cur.execute(truncate_query)
# conn.commit()

In [41]:
# Inserto los mismos registros dos veces para generar más duplicados
insert_query = 'INSERT INTO covid_data VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'
data_to_insert = [tuple(row) for row in df.values]
with conn.cursor() as cur:
    cur.executemany(insert_query, data_to_insert)
    conn.commit()

In [42]:
# Cuento la cantidad de registros para validar
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM covid_data")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data: {count}")

Cantidad de registros en la tabla covid_data: 840


In [43]:
# Query para eliminar duplicados
with conn.cursor() as cur:
    # Creo una nueva tabla temporal sin duplicados
    cur.execute("CREATE TABLE covid_data_temp AS SELECT DISTINCT * FROM covid_data;")

    # Elimino la tabla original
    cur.execute("DROP TABLE covid_data;")

    # Renombro la tabla temporal como la tabla original
    cur.execute("ALTER TABLE covid_data_temp RENAME TO covid_data;")

    # Confirmo los cambios
    conn.commit()

    # Obtengo la cantidad de registros después de eliminar duplicados
    cur.execute("SELECT COUNT(*) FROM covid_data;")
    count = cur.fetchone()[0]
    print(f"Cantidad de registros en la tabla covid_data después de eliminar duplicados: {count}")

Cantidad de registros en la tabla covid_data después de eliminar duplicados: 413
